In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scanpy as sc

sc.settings.verbosity = 0

In [2]:
import GenKI as gk
from GenKI.preprocesing import build_adata
from GenKI.dataLoader import DataLoader
from GenKI.train import VGAE_trainer
from GenKI import utils

%load_ext autoreload
%autoreload 2

In [3]:
# example

data_dir = os.path.join("data", "filtered_gene_bc_matrices")
counts_path = str(os.path.join(data_dir, "X.txt"))
gene_path = str(os.path.join(data_dir, "g.txt"))
cell_path = str(os.path.join(data_dir, "c.txt"))

adata = build_adata(counts_path, 
                    gene_path, 
                    cell_path, 
                    meta_cell_cols=["cell_type"], # colname of cell type
                    delimiter=',', # X.txt
                    transpose=True, # X.txt
                    log_normalize=True,
                    scale_data=True,
                   )

adata = adata[:100, :300].copy()
print(adata)

load counts from data\filtered_gene_bc_matrices\X.txt
AnnData object with n_obs × n_vars = 100 × 300
    obs: 'cell_type'
    uns: 'log1p'
    layers: 'raw', 'norm'


In [4]:
# gene to ko
adata.var_names[66]

'LAMB3'

In [5]:
# load data

data_wrapper =  DataLoader(
                adata, # adata object
                target_gene = [66], # KO gene name/index
                target_cell = None, # obsname for cell type, if none use all
                obs_label = "cell_type", # colname for genes
                GRN_file_dir = "GRNs", # folder name for GRNs
                rebuild_GRN = True, # whether build GRN by pcNet
                pcNet_name = "pcNet_example", # GRN file name
                verbose = True, # whether verbose
                n_cpus = 8, # multiprocessing
                )

data_wt = data_wrapper.load_data()
data_ko = data_wrapper.load_kodata()

use all the cells (100) in adata
build GRN
ray init, using 8 CPUs
execution time of making pcNet: 6.34 s
GRN has been built and saved in "GRNs\pcNet_example.npz"
init completed



In [6]:
# init trainer

hyperparams = {"epochs": 100, 
               "lr": 7e-4, 
               "beta": 1e-4, 
               "seed": 8096}
log_dir=None 

sensei = VGAE_trainer(data_wt, 
                     epochs=hyperparams["epochs"], 
                     lr=hyperparams["lr"], 
                     log_dir=log_dir, 
                     beta=hyperparams["beta"],
                     seed=hyperparams["seed"],
                     verbose=True,
                     )

In [7]:
# %%timeit
sensei.train()

Epoch: 000, Loss: 1.5318, AUROC: 0.8618, AP: 0.7514
Epoch: 001, Loss: 1.4803, AUROC: 0.8643, AP: 0.7555
Epoch: 002, Loss: 1.5022, AUROC: 0.8668, AP: 0.7596
Epoch: 003, Loss: 1.4556, AUROC: 0.8692, AP: 0.7634
Epoch: 004, Loss: 1.4764, AUROC: 0.8715, AP: 0.7671
Epoch: 005, Loss: 1.4815, AUROC: 0.8739, AP: 0.7708
Epoch: 006, Loss: 1.4358, AUROC: 0.8760, AP: 0.7744
Epoch: 007, Loss: 1.4775, AUROC: 0.8783, AP: 0.7781
Epoch: 008, Loss: 1.4364, AUROC: 0.8804, AP: 0.7812
Epoch: 009, Loss: 1.4689, AUROC: 0.8823, AP: 0.7839
Epoch: 010, Loss: 1.4504, AUROC: 0.8842, AP: 0.7863
Epoch: 011, Loss: 1.4228, AUROC: 0.8859, AP: 0.7886
Epoch: 012, Loss: 1.4194, AUROC: 0.8875, AP: 0.7907
Epoch: 013, Loss: 1.3903, AUROC: 0.8890, AP: 0.7928
Epoch: 014, Loss: 1.3900, AUROC: 0.8904, AP: 0.7948
Epoch: 015, Loss: 1.3866, AUROC: 0.8916, AP: 0.7969
Epoch: 016, Loss: 1.3610, AUROC: 0.8928, AP: 0.7991
Epoch: 017, Loss: 1.3729, AUROC: 0.8940, AP: 0.8016
Epoch: 018, Loss: 1.3424, AUROC: 0.8953, AP: 0.8042
Epoch: 019, 

In [ ]:
# sensei.save_model('model_example')

In [8]:
# get distance between wt and ko

z_mu_wt, z_std_wt = sensei.get_latent_vars(data_wt)
z_mu_ko, z_std_ko = sensei.get_latent_vars(data_ko)
dis = gk.utils.get_distance(z_mu_ko, z_std_ko, z_mu_wt, z_std_wt, by="KL")

In [9]:
# raw ranked gene list

res_raw = utils.get_generank(data_wt, dis, rank=True)
res_raw.head()

,dis,rank
LAMB3,52.747102,1
COL3A1,0.138186,2
S100A9,0.085078,3
MDK,0.084988,4
MFAP2,0.079943,5


In [10]:
# if permutation test

null = sensei.pmt(data_ko, n=100, by="KL")
res = utils.get_generank(data_wt, dis, null,)
#                       save_significant_as = 'gene_list_example')
res

Permutating: 100%|██████████| 100/100 [00:02<00:00, 33.71it/s]


,dis,index,hit,rank
LAMB3,52.747102,66,100,1
COL3A1,0.138186,9,100,2
S100A9,0.085078,194,100,3
TIMP1,0.033647,145,100,4
